# Cabeçalho - TODO (escrever bonitinho)

## Resumo do trabalho - TODO (escrever bonitinho)

## Importando bibliotecas

**TODO - dando xabu na matplotlib e xgboost**

In [ ]:
%matplotlib inline
%lsmagic

import time
import math

import numpy as np
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

#import xgboost as xgb

## Definindo funções úteis

In [ ]:
def graficos_boxplot(X, colunas=None, grupo=None, restricoes=None):
    
    if grupo == None:
        print ('nenhum grupo informado')
        return None
    
    if colunas == None: colunas = X.columns.tolist()
    
    dado = X[colunas + [grupo]].copy()
    
    for col in colunas:
        plt.figure(figsize=(16, 4))
        ax = sns.boxplot(x=grupo, y=col, data=dado)
        plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k")
        plt.xticks(rotation=45)
    return

def detectar_outliers(X, colunas=None, metodo='one class svm', contamination=0.1):
    
    if colunas == None: colunas = X.columns.tolist()
    
    x_local = X[colunas].copy()
    
    col_num = x_local.select_dtypes(include = ['float64', 'int64']).columns.tolist()
    col_cat = x_local.select_dtypes(include = ['object']).columns.tolist()
    
    for col in col_cat:
        enc = LabelEncoder()
        x_local[col] = enc.fit_transform(x_local[col].astype(str))
        
    for col in col_num:
        x_local[col].fillna(0, inplace=True)
        
    x_local['is outlier'] = 0
    
    dado = x_local[colunas]
    
    isolation_forest = IsolationForest(n_estimators=100, behaviour='new', contamination=contamination)
    robust_covariance = EllipticEnvelope(contamination=contamination)
    OneClassSVM = svm.OneClassSVM(kernel='poly', degree=2, gamma=1000, tol=0.1, nu=contamination)
    local_out_factor = LocalOutlierFactor(n_neighbors=35, contamination=contamination)
    
    if metodo == 'one class svm': out = OneClassSVM
    if metodo == 'isolation forest': out = isolation_forest
    if metodo == 'robust covariance': out = robust_covariance
    if metodo == 'local outlier fraction': out = local_out_factor
    
    if metodo == 'local outlier fraction':
        dado['is outlier'] = out.fit_predict(dado)
    else:
        out.fit(dado)
        dado['is outlier'] = out.predict(dado)
    
    mask = dado[dado['is outlier'] == -1].index.tolist()
    x_local.loc[mask, 'is outlier'] = 1
    
    return x_local['is outlier']

def preencher_buracos(X, coluna, valor = None, coluna_grupo = None):
    if valor != None: 
        X[coluna].fillna(valor, inplace=True)
    elif coluna_grupo == coluna:
        X[coluna].fillna(X[coluna].mean())
    else:
        mask = X[coluna].isnull()
        replace_dict = X[coluna, coluna_grupo].groupby(coluna_grupo).mean().to_dict()[coluna]
        X.loc[mask, coluna] = X[coluna_grupo][mask].replace(replace_dict)
    return X

def normalizar_numericas(X, colunas_numericas = None):
    if colunas_numericas == None: colunas_numericas = X.columns.tolist()
    scaler = RobustScaler()
    X[colunas_numericas] = scaler.fit_transform(X[colunas_numericas])
    return X, scaler

def codificar_categoricas(X, colunas_categoricas = None):
    if colunas_categoricas == None: colunas_categoricas = X.columns.tolist()
    colunas_numericas = [x for x in X.columns.tolist() if x not in colunas_categoricas]
    enc = OneHotEncoder(handle_unknown='ignore')
    X_cat = pd.DataFrame(enc.fit_transform(X[colunas_categoricas]).toarray())
    X.reset_index(inplace=True, drop=True)
    res = pd.concat([X_cat, X[colunas_numericas]], axis=1)
    return res, enc

def fitar_modelo(modelo, X, y, X_test = None, y_test = None):
    modelo.fit(X, y)
    train_score = r2_score(y, modelo.predict(X))
    test_score = r2_score(y_test, modelo.predict(X_test))
    if X_test == None:
        return modelo, train_score
    else:
        return modelo, train_score, test_score

def importancia_das_features(X, y, colunas_numericas = None, colunas_categoricas = None, nome_variavel = ''): 
    
    x_local = X.copy()
    y_local = y.copy()
    
    if colunas_numericas != None: x_local, _ = normalizar_numericas(x_local, colunas_numericas)
    if colunas_categoricas != None:
        for col in colunas_categoricas:
            enc = LabelEncoder()
            x_local[col] = enc.fit_transform(x_local[col].astype(str))
    
    features_names = x_local.columns
    
    rf_importance = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    rf_importance.fit(x_local, y_local)
    
    feature_importances = list(rf_importance.feature_importances_)
    list_tuples = zip(features_names, feature_importances)
    
    plt.figure(123)
    plt.figure(figsize=(5, 10))
    features_names_sort, feature_importances_sort = zip(*sorted(list_tuples, key=lambda x:x[1], reverse=False)[:50])
    plt.barh(range(len(feature_importances_sort)), feature_importances_sort)
    plt.yticks(np.arange(len(features_names_sort)), features_names_sort)
    plt.title('Feature Importances - ' + nome_variavel)
    plt.show()
    
    return features_names_sort, feature_importances_sort

def calcular_correlação(X, colunas_numericas=None, colunas_categoricas=None):
    
    x_local = X.copy()
    colunas = []
    
    if colunas_numericas != None:
        x_local, _ = normalizar_numericas(x_local, colunas_numericas)
        colunas = colunas + colunas_numericas
    if colunas_categoricas != None:
        colunas = colunas + colunas_categoricas
        for col in colunas_categoricas:
            enc = LabelEncoder()
            x_local[col] = enc.fit_transform(x_local[col].astype(str))
    
    correlacao = {}
    for col in colunas:
        val = x_local.drop(col, axis=1).apply(lambda x: x.corr(x_local[col]))
        correlacao[col] = val.sort_values(ascending=False)
    
    df = pd.DataFrame(correlacao)
            
    return df

def selecionar_features(X, y, y_name=None, colunas_numericas=None, colunas_categoricas=None, limite_corr=0.8, limite_importancia=0.8, quantidade=999):
    
    x_local = X.copy()
    y_local = y.copy()
    
    if colunas_numericas != None and colunas_categoricas != None:
        corr = calcular_correlação(x_local, colunas_numericas=colunas_numericas, colunas_categoricas=colunas_categoricas)
        features_names_sort, feature_importances_sort = importancia_das_features(x_local, y_local, colunas_numericas=colunas_numericas, colunas_categoricas=colunas_categoricas, nome_variavel = y_name)
    elif colunas_numericas != None and colunas_categoricas == None:
        corr = calcular_correlação(x_local, colunas_numericas=colunas_numericas)
        features_names_sort, feature_importances_sort = importancia_das_features(x_local, y_local, colunas_numericas=colunas_numericas, nome_variavel = y_name)
    elif colunas_numericas == None and colunas_categoricas != None:
        corr = calcular_correlação(x_local, colunas_categoricas=colunas_categoricas)
        features_names_sort, feature_importances_sort = importancia_das_features(x_local, y_local, colunas_categoricas=colunas_categoricas, nome_variavel = y_name)
    else:
        print ("algo errado ai meu camarada")
        return None
    
    features = corr.columns.tolist()
    features_slct = []
    count = 0
    for f, v in zip(reversed(features_names_sort), reversed(feature_importances_sort)):
        print (f, v)
        if count > quantidade: break
        add = True
        if float(v) > limite_importancia: add = False
        for f_f in features_slct:
            if v < 0.001 or abs(corr.loc[f, f_f]) > limite_corr: add = False
        if add: 
            features_slct.append(f)
            count += 1
    
    x_features_slct = x_local[features_slct]
    return x_features_slct, features_slct

def busca_aleatoria(X, y, modelo, parametros, metrica, n_folds = 10, n_iter = 200):
    
    start_time = time.time()
    
    print ("\n \n buscando melhores parâmetros...")
    
    otm_model = RandomizedSearchCV(modelo, param_distributions=parametros, n_iter=n_iter, scoring=metrica, cv=n_folds, n_jobs=1)
    otm_model.fit(X, y)
    
    melhores_parametros = otm_model.best_params_
    melhor_modelo = otm_model.best_estimator_
    
    resultados = pd.DataFrame(otm_model.cv_results_)
    resultados.sort_values(by='rank_test_score', inplace=True)
    resultados.sort_values(by='rank_test_score', inplace=True)
    print ("melhores parâmetros encontrados")
    
    end_time = time.time()
    otm_time = (end_time - start_time) / 60
    
    best_score = otm_model.best_score_
    
    return melhor_modelo, melhores_parametros, best_score, resultados, otm_time

## Importando o dado

In [ ]:
ls data/

In [ ]:
fat_df = pd.read_csv('./data/Fat_Supply_Quantity_Data.csv')

In [ ]:
fat_df.head()

In [ ]:
protein_df = pd.read_csv('./data/Protein_Supply_Quantity_Data.csv')

In [ ]:
protein_df.head()

In [ ]:
kcal_df = pd.read_csv('./data/Food_Supply_kcal_Data.csv')

In [ ]:
kcal_df.head()

In [ ]:
quantity_kg_df = pd.read_csv('./data/Food_Supply_Quantity_kg_Data.csv')

In [ ]:
quantity_kg_df.head()

In [ ]:
description_df = pd.read_csv('./data/Supply_Food_Data_Descriptions.csv')

In [ ]:
description_df.head()

## Juntando datasets

In [ ]:
dfs = [fat_df, protein_df, kcal_df, quantity_kg_df, description_df]

In [ ]:
for df in dfs:
    print (df.shape)

In [ ]:
cols = {}
i = 0
for df in dfs[:-1]:
    i += 1
    cols['df'+str(i)] = list(df.columns)
df_cols = pd.DataFrame(cols)

In [ ]:
df_cols

In [ ]:
shared_cols = ['Country', 'Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Population', 'Unit (all except Population)']

In [ ]:
dataset = fat_df[shared_cols].copy()

In [ ]:
dataset.head()

In [ ]:
cols = list(fat_df.columns)

In [ ]:
cols = ['Country'] + [i for i in cols if i not in shared_cols]

In [ ]:
fat_df = fat_df[cols].copy()

In [ ]:
protein_df = protein_df[cols].copy()

In [ ]:
kcal_df = kcal_df[cols].copy()

In [ ]:
quantity_kg_df = quantity_kg_df[cols].copy()

In [ ]:
fat_cols_map = {i:i+'_fat' for i in cols if i != 'Country'}

In [ ]:
protein_cols_map = {i:i+'_protein' for i in cols if i != 'Country'}

In [ ]:
kcal_cols_map = {i:i+'_kcal' for i in cols if i != 'Country'}

In [ ]:
quant_cols_map = {i:i+'_quantity' for i in cols if i != 'Country'}

In [ ]:
fat_df.rename(fat_cols_map, axis=1, inplace=True)

In [ ]:
protein_df.rename(protein_cols_map, axis=1, inplace=True)

In [ ]:
kcal_df.rename(kcal_cols_map, axis=1, inplace=True)

In [ ]:
quantity_kg_df.rename(quant_cols_map, axis=1, inplace=True)

In [ ]:
dataset = dataset.merge(fat_df, how='left', on='Country', copy=False)

In [ ]:
dataset = dataset.merge(protein_df, how='left', on='Country', copy=False)

In [ ]:
dataset = dataset.merge(kcal_df, how='left', on='Country', copy=False)

In [ ]:
dataset = dataset.merge(quantity_kg_df, how='left', on='Country', copy=False)

In [ ]:
dataset.shape

## Analisando o dado

In [ ]:
dataset.info()

In [ ]:
dataset.dtypes.value_counts()

In [ ]:
data_num = dataset.select_dtypes(include = ['float64', 'int64'])
data_cat = dataset.select_dtypes(include = ['object'])

In [ ]:
data_cat.head()

In [ ]:
data_num.describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
for col, val in zip(dataset.isnull().sum().index.to_list(), list(dataset.isnull().sum().values)):
    if val > 0:
        print (col, val) 

In [ ]:
dataset = dataset[dataset['Confirmed'].isnull() != True]

In [ ]:
dataset.shape

In [ ]:
for col, val in zip(dataset.isnull().sum().index.to_list(), list(dataset.isnull().sum().values)):
    if val > 0:
        print (col, val) 

In [ ]:
dataset['Undernourished'].unique()

In [ ]:
dataset.loc[dataset['Undernourished'] == '<2.5', 'Undernourished'] = '2.5'

In [ ]:
dataset['Undernourished'] = dataset['Undernourished'].astype(float)

In [ ]:
dataset['Obesity'].unique()

In [ ]:
dataset['Obesity'] = dataset['Obesity'].astype(float)

In [ ]:
dataset['Obesity'].fillna(dataset['Obesity'].mean(), inplace=True)

In [ ]:
dataset['Undernourished'].fillna(dataset['Undernourished'].mean(), inplace=True)

In [ ]:
dataset.isnull().sum().unique()

## Golden features

In [ ]:
targets = ['Confirmed', 'Deaths', 'Recovered', 'Active']

In [ ]:
data_num = dataset.select_dtypes(include = ['float64', 'int64'])
for t in targets:
    data_corr = data_num.corr()[t]
    golden_features_list = data_corr[abs(data_corr) > 0.2].sort_values(ascending=False)
    print (t, golden_features_list)

**TODO: fazer uns gráficos das correlações (golden features)**

## Remover outliers

In [ ]:
colunas_outlier = data_num.columns.to_list()

dataset['is outlier'] = detectar_outliers(dataset, colunas=colunas_outlier, metodo='local outlier fraction', 
                                          contamination=0.01)
print (str(list(dataset['is outlier']).count(1)) + ' outliers detectados')
dataset = dataset[dataset['is outlier']==0]

## Separando features das targets

In [ ]:
targets = ['Obesity', 'Undernourished', 'Confirmed', 'Deaths', 'Recovered', 'Active']
y_dict = {}
for t in targets:
    y_dict[t] = dataset[t]

## Feature engineering

In [ ]:
cols = dataset.columns.to_list()

In [ ]:
features = [f for f in cols if f not in targets]

x_features = dataset[features].copy()

In [ ]:
x_features.head()

In [ ]:
del x_features['Unit (all except Population)']

In [ ]:
x_features.dtypes.value_counts()

In [ ]:
data_num = x_features.select_dtypes(include = ['float64', 'int64'])
data_cat = x_features.select_dtypes(include = ['object'])
colunas_a_normalizar = data_num.columns.tolist()
colunas_a_codificar = data_cat.columns.tolist()

In [ ]:
features_importances_dict = {}
features_names_dict = {}
for t in targets:
    features_names_sort, feature_importances_sort = importancia_das_features(x_features, y_dict[t], colunas_numericas=colunas_a_normalizar, colunas_categoricas=colunas_a_codificar, nome_variavel = t)
    features_importances_dict[t] = feature_importances_sort
    features_names_dict[t] = features_names_sort

**TODO: colorir gráfico de acordo com sinal (negativo ou positivo) da correlação**